In [1]:
print("Lecture-29")

Lecture-29


## Geri Beslemeli Ağlarda Çıktı Üretimi

Biz şimdiye kadar örneklerimizde gelecek kestirimi yapmaya çalıştık. Ancak bazn bu gelecek tahminleri çıktı üretimleri için de kullaınlabilmektedir. Özellikle metinlerin üretiminde 2015 yılından itibaren LSTM tabanlı geri beslemeli ağlar kullanılmaya başlanmıştır.

Gerçekleştirme Adımları:

1) Üretilecek metni belli bir kişinin tarzına uydurmak için o kişinin yazıları elde edilir.

2) Sonra bir geri beslemeli sinir ağı oluştururarak sinir ağı yazılarla eğitilir. Burada karakter temelli bir çıktı üretiminin söz konusu olduğunu düşündelim. Bunun için seçilen kişiye ilişkin yazıların belli miktradaki karakterden oluşan kısmları - örneğin 32 karakter olduğunu düşünelim - ve bu kusumdan sonra gelen karakterler eğitim amacıyla toplanacaktır. Böylece bu işlemden 32 karakterden oıluşan bir yazı parçaları ile (dataset_x), 1 karakterden oluşan sonuç değerleri (dataset_y) elde edilecektir. 

3) Geri beslemeli bir yapay sinir ağı modeli kurularak ağ oluşturulan veri kümesiyle eğitilir. Geri besleme için genelllikle LSTM modeli tercih edilmektedir.

4) Artık elimizde 32 karakterini girdi olarak verdiğimiz bir metnin 33. karkaterini tahmin edebilen bir geri beslemeli yapay sinir ağı modeli vardır. Bu durumda biz 32 karakterlik bir yazıyı başlangıç yazısı olarak verirsek ağdan bunun 33. karakterini elde edebiliriz. Bunu sonra yine bu 33 karakterli yazının son 32 karakterini alarak yeni bir karakter elde ederiz. Bu işlemi böyle sürdürürsek istediğimiz uzunlukta bir yazı elde etmiş oluruz.

5) Modelin bu biçimde oluşturulması yazının tek düze ve yaratıcılık içermeyen bir yapıda olmasına yol açabilmektedir. Bu nedenle her dfefasında ağdan elde edilen karakteri doğrudan kullanmak yerine işin içine bir miktar rassallık da katarak belli bir olasılığı dağılımı çerçevesinde yeni karakterin elde edilmesi daha yaratıcı bir metnin oluşturulmasına yol açacaktır.

Örnek: Belli bir bestecinin tarzına göre beste yapan bir sinir ağı modeli oluşturulabilir.

Uygulamalı Örnek: Nietzsche 'nin metinleri

In [2]:
import tensorflow.keras

path = tensorflow.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

with open(path) as f:
    text = f.read().lower()

AlreadyExistsError: Another metric with the same name already exists.

In [ ]:
import tensorflow.keras

path = tensorflow.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

with open(path) as f:
    text = f.read().lower()

print(text)

# yazıda 3'er atlamalı olarak 60'ar karakterden oluşan parçaları sentences isimli bir listeye yerleştireceğiz.
TEXT_LENGTH = 60
STEP = 3

# yapay sinir ağı 60 karakterden sonraki 61. karakteri tahmin edeceğinden dolayı bu 60 karakterden sonraki karakteri de next_char isimli bir listede toplayacağız.
sentences = []
next_chars = []

for i in range(0, len(text) - TEXT_LENGTH, STEP):
  sentences.append(text[i:i + TEXT_LENGTH])
  next_chars.append(text[i + TEXT_LENGTH])

print("Number of sequences: {}".format(len(sentences)))

# bundan sonra yazı içerisinde farklı kaç tane karakter olduğunu tespit edip bu karakterleri de bir listeye yerleştirerek
# koda devam edelim.
chars = sorted(list(set(text)))
print('Number of unique chars: {}'.format(len(chars)))

chars

# Yazı parçalarındaki karakterlerinin her birini "one hot encoding" biçiminde oluşturmarmız gerekir. Zira eğer biz akrakter kodlarını
# doğrudan kullanırsak model sanki sıralı (ordinal) bir ölçek söz konusuymuş gibi davranacaktır. OHE işlemini kolaylaştırmak için bir
# sözlük oluşturabiliriz. Bu sözlük sayesinde yazı içerisindeki bir karakter verildiğinde onun indeksini hızlı bir biçimde elde edeceğiz.

char_dict = {char: index for index, char in enumerate(chars)}

char_dict

# Bu sözlük sayesinde biz karakteri verdiğimizde ona karşı gelen sayısal değeri alabileceğiz. Artık sıra training_dataset_x ve
# training_dataset_y nesnelerinin ndarray olarak oluşturulmasında. Bunun için önce bu diziler içnide sıfır olacak biçimde oluşturualım
import numpy as np
training_dataset_x = np.zeros((len(sentences), TEXT_LENGTH, len(chars)), dtype=np.uint8)
training_dataset_y = np.zeros((len(sentences), len(chars)), dtype=np.uint8)

# Yalnızca uygun elemanı 1 yaparak one hot encoding vektörlerini kolay bir biçimde oluştuırabiliriz.

for i, sentence in enumerate(sentences):
  for k, char in enumerate(sentence):
    training_dataset_x[i, k, char_dict[char]] = 1
  training_dataset_y[i, char_dict[next_chars[i]]] = 1

# artık training_dataset_x ve training_dataset_y one hot encoding biçiminde oluşturulmuş durumdadır.

training_dataset_y

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

model = Sequential(name = 'Nietzsche-LSTM')
model.add(LSTM(128, activation='tanh', input_shape=(TEXT_LENGTH, len(chars)), name = 'LSTM'))
model.add(Dropout(0.3, name = 'Dropout'))
model.add(Dense(len(chars), activation='softmax', name = 'Output'))

model.summary()

# Çıktı matnanının toplamda len(chars) uzunlukta nöronlardan oluştuğunu görüyoruz. Çıktı katmanının aktivasyon fonksiyonu "softmax"
# olduuğna göre tüm çıktı nöronlarının toplam değeri 1 olacaktır. Tabii biz bunlar arasından en yükske değeri tahmin olarak alacağız.
# Ağımızın optimizasyon algoritması "rmsprop" olarak loss fonksiyonun da daha önceden benzer sıınflandırma örneklerinde olduğu gibi
# "categorical_crossentropy" olarak alınmıştır.

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

esc = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
hist = model.fit(training_dataset_x, training_dataset_y, batch_size=32, epochs=20, validation_split=0.2, callbacks=[esc])

import matplotlib.pyplot as plt

figure = plt.gcf()
figure.set_size_inches((15, 5))
plt.title('Loss - Epoch Graphics')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(1, len(hist.history['loss']) + 1), hist.history['loss'])
plt.plot(range(1, len(hist.history['val_loss']) + 1), hist.history['val_loss'])
plt.legend(['Loss', 'Validation Loss'])
plt.show()


figure = plt.gcf()
figure.set_size_inches((15, 5))
plt.title('Categorical Accuracy - Epoch Graphics')
plt.xlabel('Epoch')
plt.ylabel('Categorical Accuracy')
plt.plot(range(1, len(hist.history['categorical_accuracy']) + 1), hist.history['categorical_accuracy'])
plt.plot(range(1, len(hist.history['val_categorical_accuracy']) + 1), hist.history['val_categorical_accuracy'])
plt.legend(['Categorical Accuracy', 'Validation Categorical Accuracy'])
plt.show()

# Şimdi de belli bir TEXT_LENGTH uzunluuğnda yazı alıp bir döngü içerisinde ona 400 karakter ekleyerek yeni bir yazı üretelim.
# TEXT_LENGTH uzunluğundaki yazıyı verip bundan elde edilecek TEXT_LENGTH + 1. karakteri tahmin ederiz. Sonra son TEXT_LENGTH uzunlukta
# yazıyı verip sonraki karakteri tahmin ederiz. Böylece 400 karakteri bir döngü içerisinde bu biçimde elde ederiz. Örneğimizde ağ
# tarafından elde edilen karkaterlerin olasılık dağılımına göre biraz değiştirilmesi gerebeiblmektedir. Yani üretilen karakterin değil de ona yakın
# bir karakterin seçilmesi tekdüzeliği engellemek bakımından uygun olabilir.

def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)

  return np.argmax(probas)

# Programın yazı üretim kısmı için TEXT_LENGTH uzunluktaki yazıları ohe biçimine dönüştürmemiz gerekir.

def str_to_ohe(s):
  ohe = np.zeros(shape=(1, TEXT_LENGTH, len(chars)))
  for index, char in enumerate(s):
    ohe[0, index, char_dict[char]] = 1.0

  return ohe

# Şimdi rastgele bir TEXT_LENGTH uzunluuğunda bir yazı oluşturalım.

r = np.random.randint(len(sentences) - TEXT_LENGTH - 1)
random_initial_sentence = text[r: r + 60]

r

random_initial_sentence

# Şimdi de 400 karakterlik bir yazı üretelim.

generated_text = random_initial_sentence
for i in range(400):
  ohe = str_to_ohe(generated_text[i: TEXT_LENGTH + i])
  result = model.predict(ohe)[0]
  char = chars[sample(result, 0.2)]
  generated_text += char

print(f'[{generated_text[0:TEXT_LENGTH]}]{generated_text[TEXT_LENGTH:]}')